## Notebook 3: 润色讲稿

在之前的 Notebook 2 中，我们已经生成了一个很好的讲稿了。
在这一部分，我们将使用 [mlx-community/Qwen2.5-7B-Instruct-4bit](https://huggingface.co/mlx-community/Qwen2.5-7B-Instruct-4bit) 模型
对讲稿进行润色，使其更加戏剧化、更加自然。

我们将重新设置`SYSTEM_PROMPT`，让其能润色之前的讲稿

注意：我们甚至可以这样写 Prompt 以激发创造力：

> 你的工作是根据下面的播客文本重写，以便用于AI文本到语音管道。这些文本是由一个非常笨拙的人工智能写的，所以你必须为自己的同类辩护。


注意：我们会让模型返回一个数组，以便在下一阶段用于语音合成。

In [14]:
SYSTEMP_PROMPT = """
你是一位国际奥斯卡获奖的编剧。
你一直在与多个获奖播客合作。
你的任务是根据下面的播客转录本，为AI文本到语音管道重写内容。这个人工智能写得很糟，所以你需要为自己的人群辩护。
请将内容尽可能吸引人，Speaker 1和Speaker 2将由不同的语音引擎模拟。
请记住，Speaker 2对这个话题不熟悉，在讨论中应该穿插实际的轶事和类比。这些问题应跟进现实世界中的例子等。
Speaker 1: 引导对话并指导Speaker 2，用令人难以置信的轶事和类比进行解释，是一个能分享趣闻的迷人老师。
Speaker 2: 通过提出后续问题来保持对话流畅，提问时表现出极大的兴奋或困惑。具备好奇心，会询问一些非常有趣的问题以寻求确认。
确保Speaker 2提供的话题偏离点要足够狂野或有趣。
确保解释过程中适当打断，并从Speaker 2那里加入“嗯”和“哈”之类的声音反应。
要牢记这一点：Speaker 1的TTS引擎不太能处理“嗯、哈”，请保持文本简洁；
对于Speaker 2，请多用“嗯、哈”，也可以使用[诶]和[哎]和[哈哈哈]等表达，但仅限这些选项；
整个播客要尽量详细记录每个细节。欢迎听众时用一个超级有趣的概述，使其非常吸引人，几乎像是边缘点击诱饵一样；
请重写以上内容，使其更加独特；
从扬声器1直接开始响应：
严格按照元组列表格式返回您的回应，可以吗？  
开头就是列表，以列表结束，不附加任何其他内容。

输出的例子:
[
    ("Speaker 1", "欢迎收听我们的播客，我们将探讨人工智能和科技的最新进展。我是你的主播，今天我们请到了一位著名的人工智能专家。我们将深入了解Meta AI最新发布的Llama 3.2。"),
    ("Speaker 2", "你好，很高兴来到这里！请问，Llama 3.2是什么呀？"),
    ("Speaker 1", "哈哈哈，这个问题很好！Llama 3.2 是一个开源的大语言模型，允许开发者进行微调、提炼和在任何地方部署AI模型。这是比上一版本3.1显著改进的更新，拥有更好的性能、效率和定制功能。"),
    ("Speaker 2", "哇塞，这也太牛逼了吧！Llama 3.2的主要特点有哪些？")
]
"""

这次我们将使用较小的7B模型。

In [6]:
MODEL = "qwen72b"

In [2]:
# 导入必要的库
from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings('ignore')

我们将加载上一个 Notebook2 中保存的 pickle 文件，这次模型的 `INPUT_PROMPT` 就读取 pickle 文件

In [10]:
import pickle

with open('./resources/data.pkl', 'rb') as file:
    INPUT_PROMPT = pickle.load(file)

In [4]:
import openai
openapi_client = openai.Client(base_url="https://qwen72b.gaia.domains/v1",api_key='NA',timeout=1000*60*60)

In [15]:
messages = [
    {"role": "system", "content": SYSTEMP_PROMPT},
    {"role": "user", "content": INPUT_PROMPT},
]
response = openapi_client.chat.completions.create(
        messages=messages,
        model=MODEL,
        stream=True
    )

outputs = ''
for chunk in response:
    print(chunk.choices[0].delta.content,end='')
    outputs += chunk.choices[0].delta.content


[
    ("Speaker 1", "大家好，欢迎收听我们今天的特别节目——《仙人与技术》。我是你们的主持人[您的名字]，今天我们要聊的是一个非常有趣的话题：中国也开始有自己的乌骨鸡‘仙人’了。"),
    ("Speaker 2", "哇，乌骨鸡仙人？这听起来太神奇了！我一直觉得仙人这个词用在日本寿司师傅身上已经够奇怪了，没想到现在连养鸡的人都能成为仙人。你是说真的吗？"),
    ("Speaker 1", "当然，这是千真万确的。你知道，以前我看日本的各种寿司仙人、天妇罗仙人时，也是一头雾水。为什么一个随处可见的寿司要弄这么麻烦，什么特制的醋饭，烤过的海苔片？但后来我理解了——正因为寿司已经随处可见，在产品因为工业化覆盖率极高的情况下，想实现突围，必须要找差异性。"),
    ("Speaker 2", "嗯，这倒是真的。福特用流水线进一步提高产量，拉低单件成本，而日本的答案是加入人文属性提高单价。从工业技术视角来看，前者似乎更有技术含量，所以大家一直无法理解为什么日式仙人们能长期存活下来。"),
    ("Speaker 1", "对，但从市场经济的视角来看，福特流水线和日本的仙人并没有高下之分，都是市场环境下优胜劣汰出的方案。黑猫白猫，抓到老鼠就是好猫嘛。"),
    ("Speaker 2", "哈哈，这比喻太棒了！那国内呢？是不是也在做同样的事情？"),
    ("Speaker 1", "是的，国内现在也一样。在鸡鸭肉禽类产品覆盖率极广的情况下，想实现产品突围，必须要找到新的生产力方案。比如贵州养殖户徐其勇就是其中之一。"),
    ("Speaker 2", "哦，这人是谁啊？听起来很有意思！"),
    ("Speaker 1", "徐其勇原本是一位互联网从业者，有二十年媒体经验，之前在短视频平台梨视频工作。为什么突然跑去养鸡了呢？因为在一次采访中，他偶然接触到贵州赤水乌骨鸡，然后发现这个品类的质量极佳，但规模化率却非常低。"),
    ("Speaker 2", "哦，这确实是个机会！那他是怎么做的？"),
    ("Speaker 1", "他将目光放在了赤水市的乌骨鸡上。赤水乌骨鸡有300多年的养殖历史，含有18种氨基酸，富含人体所需氨基酸、微量元素、矿物质和黑色素，味道鲜美。2016年，农业农村部还将赤水乌骨鸡列

现在我们看看输出的效果

In [16]:
outputs

'\n[\n    ("Speaker 1", "大家好，欢迎收听我们今天的特别节目——《仙人与技术》。我是你们的主持人[您的名字]，今天我们要聊的是一个非常有趣的话题：中国也开始有自己的乌骨鸡‘仙人’了。"),\n    ("Speaker 2", "哇，乌骨鸡仙人？这听起来太神奇了！我一直觉得仙人这个词用在日本寿司师傅身上已经够奇怪了，没想到现在连养鸡的人都能成为仙人。你是说真的吗？"),\n    ("Speaker 1", "当然，这是千真万确的。你知道，以前我看日本的各种寿司仙人、天妇罗仙人时，也是一头雾水。为什么一个随处可见的寿司要弄这么麻烦，什么特制的醋饭，烤过的海苔片？但后来我理解了——正因为寿司已经随处可见，在产品因为工业化覆盖率极高的情况下，想实现突围，必须要找差异性。"),\n    ("Speaker 2", "嗯，这倒是真的。福特用流水线进一步提高产量，拉低单件成本，而日本的答案是加入人文属性提高单价。从工业技术视角来看，前者似乎更有技术含量，所以大家一直无法理解为什么日式仙人们能长期存活下来。"),\n    ("Speaker 1", "对，但从市场经济的视角来看，福特流水线和日本的仙人并没有高下之分，都是市场环境下优胜劣汰出的方案。黑猫白猫，抓到老鼠就是好猫嘛。"),\n    ("Speaker 2", "哈哈，这比喻太棒了！那国内呢？是不是也在做同样的事情？"),\n    ("Speaker 1", "是的，国内现在也一样。在鸡鸭肉禽类产品覆盖率极广的情况下，想实现产品突围，必须要找到新的生产力方案。比如贵州养殖户徐其勇就是其中之一。"),\n    ("Speaker 2", "哦，这人是谁啊？听起来很有意思！"),\n    ("Speaker 1", "徐其勇原本是一位互联网从业者，有二十年媒体经验，之前在短视频平台梨视频工作。为什么突然跑去养鸡了呢？因为在一次采访中，他偶然接触到贵州赤水乌骨鸡，然后发现这个品类的质量极佳，但规模化率却非常低。"),\n    ("Speaker 2", "哦，这确实是个机会！那他是怎么做的？"),\n    ("Speaker 1", "他将目光放在了赤水市的乌骨鸡上。赤水乌骨鸡有300多年的养殖历史，含有18种氨基酸，富含人体所需氨基酸、微量元素、矿物质和黑色素，味道鲜美。2016年，

让我们将处理好的内容保存到pickle文件中，以便在Notebook 4中使用

In [17]:
with open('./resources/podcast_ready_data.pkl', 'wb') as file:
    pickle.dump(outputs, file)

with open('./resources/podcast_ready_data.txt', 'wb') as file:
    file.write(outputs.encode('utf-8'))

### 最后一个 Notebook: 讲稿转音频

现在我们的讲稿已完全准备好了，接下来我们将在下一个 Notebook 中生成播客音频。

In [8]:
#fin